# DCGAN Implementation

Implementation of the (vanilla) Deep Convolutional Generative Adversarial Network defined by <a href="https://arxiv.org/pdf/1511.06434.pdf">Radford, Metz, and Chintala (2016)</a>.  Testing is done on the <a href="https://www.cs.toronto.edu/~kriz/cifar.html">CIFAR-10</a> benchmark image dataset, stored in pickled format in the `data/` directory.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from collections import OrderedDict

## Load CIFAR Data

In [ ]:
# Helper functions to convert CIFAR-10 to Pytorch Dataset

def unpickle(file):
    import pickle
    with open(file, 'rb') as in_file:
        pickle_dict = pickle.load(in_file, encoding='bytes')
    return pickle_dict


def read_data(file_name):
    """Given path to CIFAR data batch, returns raw X, y tensors."""
    
    batch_dict = unpickle(file_name)
    X_raw = torch.tensor(batch_dict[b'data'])
    y_raw = torch.tensor(batch_dict[b'labels']).long()
    return X_raw, y_raw


def shape_image(X):
    """Reshapes raw data tensor to nn.module-compatible RGB image"""
    
    # Each row of X_raw contains RGB color channels concatenated in row-major order
    # Need to first split channels into dim 1 on tensor, then shape dim 2/3 into image
    image_size = 32*32
    X = torch.split(X.unsqueeze(dim=1), image_size, dim=2)
    X = torch.cat(X, dim=1)
    X = X.view(-1, 3, 32, 32)   # (N, channels, pixel rows, pixel cols)
    return X


def normalize(X, a=-1, b=1):
    """Normalizes data tensor to [a, b] using min-max scaling."""
    
    data_min = torch.min(X).float().item()
    data_max = torch.max(X).float().item()
    assert a < b, "Rescaled range [a, b] must have a < b"
    
    # First scale to [0, 1], then rescale to [a, b]
    X = (X - data_min) / (data_max - data_min)
    X = (X * (b - a)) + a  
    return X


class CIFARDataset(torch.utils.data.Dataset):
    """Custom Dataset class which preprocesses and stores datasets
       from CIFAR batch files.  Works for CIFAR-10 and CIFAR-100."""
    
    def __init__(self, X=None, y=None):
        self.data = X
        self.labels = y
        
    def load(self, file_list):
        # Get list of (X, y) tuples, concatenate corresponding tensors
        combined_list = [read_data(file_name) for file_name in file_list]
        X_list, y_list = list(zip(*combined_list))
        X = torch.cat(X_list, dim=0)
        y = torch.cat(y_list, dim=0)
        
        self.data = normalize(shape_image(X))
        self.labels = y
        return self
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, index):
        # Generates an (X, y) pair at given index
        return self.data[index], self.labels[index]
    
    def cuda(self):
        if torch.cuda.is_available():
            device = torch.device("cuda:0")
            self.data = self.data.to(device)
            self.labels = self.labels.to(device)
            
    def cpu(self):
        self.data = self.data.cpu()
        self.labels = self.labels.cpu()

In [ ]:
train_set = CIFARDataset().load([f"data/data_batch_{n}" for n in range(1, 6)])
test_set = CIFARDataset().load(["data/test_batch"])
cifar_meta = unpickle("data/batches.meta")

# Visualize a few random examples
def plot_image(x, ax):
    """Helper to scale and plot output from ImageDataset."""
    image = (x.squeeze().permute(1, 2, 0) + 1) / 2
    ax.imshow(image.clone().cpu().detach())
    ax.axis("off")
    
fig, ax = plt.subplots(1, 3, figsize=(18, 6))
for i in range(3):
    X, y = train_set[i*1000]
    plot_image(X, ax[i])
    ax[i].set_title(cifar_meta[b'label_names'][y.item()].decode('ascii'))
plt.suptitle("CIFAR-10 sample images")
plt.show()

## DCGAN Architecture

In [ ]:
# Define architecture for vanilla generator and discriminator
# See also:
# https://gluon.mxnet.io/chapter14_generative-adversarial-networks/dcgan.html
# https://wandb.ai/sairam6087/dcgan/reports/DCGAN-on-CIFAR-10--Vmlldzo5NjMyOQ
# https://github.com/soumith/ganhacks

def conv_block(which_model, conv_args=[], conv_kwargs={'bias': False},
               leak_slope=0.02, batch_norm=True, activation=True):
    """Performs all the operations in a single DCGAN layer in the following order:
       batch normalization -> convolution (optional) -> nonlinearity (optional)
       
       If which == 'gen', does a transpose convolution
       If which == 'dis', does a normal convolution
       Returns a list of functions in order; should unpack and fill in nn.Sequential"""
    
    funcs = []
    
    if which_model == 'gen':
        funcs.append(nn.ConvTranspose2d(*conv_args, **conv_kwargs))
        if batch_norm:
            funcs.append(nn.BatchNorm2d(conv_args[1], affine=False))
        if activation:
            funcs.append(nn.ReLU())
            
    elif which_model == 'dis':
        funcs.append(nn.Conv2d(*conv_args, **conv_kwargs))
        if batch_norm:
            funcs.append(nn.BatchNorm2d(conv_args[1], affine=False))
        if activation:
            funcs.append(nn.LeakyReLU(leak_slope))
            
    else:
        raise ValueError("Argument `which_model` is not a valid value")
        
    # Initialize the weights to match DCGAN paper?
    # Build a list of the three functions in sequence
    return funcs
    

class DCGenerator(nn.Module):
    """Deep convolutional generator which maps latent noise vector -> (32,32) RGB-channel image.
       The latent space input z is projected and convolved with many feature maps.
       Subsequent layers use only fractional-strided convolutions (no pooling).
       All hidden layers use ReLU activation, and output layer uses tanh."""
    
    def __init__(self, z_len=100):
        
        super(DCGenerator, self).__init__()
        self.z_len = z_len
        
        self.in_layer = nn.Linear(z_len, 128 * 4 * 4)
        
        # Choose kernel_size, stride, padding to double height/width for each layer
        self.conv_layers = nn.Sequential(
            #*conv_block('gen', [z_len, 128, 4, 1, 0]),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            *conv_block('gen', [128, 128, 4, 2, 1]),
            *conv_block('gen', [128, 128, 4, 2, 1]),
            *conv_block('gen', [128, 3, 4, 2, 1], batch_norm=False, activation=False),
            nn.Tanh()
        )
    
    def forward(self, z):
        # Reshape z as 4d tensor
        z = self.in_layer(z).view(-1, 128, 4, 4)
        return self.conv_layers(z)


class DCDiscriminator(nn.Module):
    """Deep convolutional discriminator which maps (32,32) RGB-channel image -> [0, 1].
       The image is passed through several convolutional layers (again, no pooling).
       All hidden layers use LeakyReLU activation, and output layer uses sigmoid."""
    
    def __init__(self, leak_slope=0.02):
        
        super(DCDiscriminator, self).__init__()
        
        # Just reverse the convolutional layers from the Generator
        self.conv_layers = nn.Sequential(
            *conv_block('dis', [3, 128, 4, 2, 1], batch_norm=False),
            *conv_block('dis', [128, 128, 4, 2, 1]),
            *conv_block('dis', [128, 128, 4, 2, 1])
        )
        
        self.out_layer = nn.Sequential(
            nn.Linear(128 * 4 * 4, 100),
            nn.LeakyReLU(leak_slope),
            nn.Linear(100, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        # Reshape discriminator output by flattening, then feed into sigmoid activation
        x = x.view(-1, torch.prod(torch.tensor(x.shape[1:])))
        return self.out_layer(x)

In [ ]:
class OnlineGenerator(nn.Module):
    def __init__(self, nz=100, nc=3, ngf=64):
        super(OnlineGenerator, self).__init__()
        self.z_len = nz
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 32 x 32
        )
    def forward(self, z):
        return self.main(z)
    
    
class OnlineDiscriminator(nn.Module):
    def __init__(self, nc=3, ndf=64):
        super(OnlineDiscriminator, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 32 x 32
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 16 x 16
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 8 x 8
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 4 x 4
            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.main(x)

In [ ]:
gen = OnlineGenerator()
dis = OnlineDiscriminator()
z_test = torch.rand(5, 100, 1, 1)
gen(z_test).shape

## DCGAN Training

The training is done using the minimax game described in the original GAN paper by <a href="https://arxiv.org/pdf/1406.2661.pdf">Goodfellow, et al (2014)</a>.  The choice of optimizer and batch size are specific to the DCGAN architecture.

In [ ]:
# Functions to train built models and evaluate convergence

def jensen_shannon(which_model, device):
    """Returns a function to compute an empirical estimate of the Jensen-Shannon
       divergence between the true data-generating distribution P_real and
       the generated distribution P_model over parallel batches x_real and x_model.
       
       This metric is optimized by a minimax operation, first maximizing over the
       discriminator weights, then minimizing over the generator weights.
       
       Returns:
       - if which_model = 'dis': returns function (D(x_model), D(x_real)) -> scalar
       - if which_model = 'gen': returns function (D(x_model)) -> scalar
         Both functions are returned in the form of minimization problems.
       - if which_model = 'full': returns non-negative version of 'dis' function.
         Use this version to calculate divergence after an epoch."""
    
    bce = nn.BCELoss()
    
    # Functions to compute respective discriminator labels
    # Consider label smoothing: draw some value near 0 or 1
    fake = lambda d: 0.1 * torch.ones(d.size()).to(device)
    real = lambda d: 0.9 * torch.ones(d.size()).to(device)
    
    if which_model == 'dis':
        #return lambda dis_model, dis_real: \
            #-torch.mean(torch.log(dis_real) + torch.log(1 - dis_model))  
        model_loss = lambda d_model: bce(d_model, fake(d_model))
        real_loss = lambda d_real: bce(d_real, real(d_real))
        return lambda d_model, d_real: model_loss(d_model) + real_loss(d_real)
        
    elif which_model == 'gen':
        # If generated samples are rejected easily, log(1 - D(x_model)) saturates
        # (stays close to 0), so use an approximation to improve early training
        #return lambda dis_model: \
            #-torch.mean(torch.log(dis_model))
        return lambda d_model: bce(d_model, real(d_model))
    
    elif which_model == 'full':
        return lambda dis_model, dis_real: \
            torch.mean(torch.log(dis_real) + torch.log(1 - dis_model))
   
    else:
        raise ValueError("Argument `which_model` is not a valid value")

        
def train_DCGAN(gen, dis, train_set, test_set,
                num_epochs=100, dg_ratio=1, batch_size=128,
                use_cuda=True, print_every=1):
    """Simultaneously trains generator and discriminator using minimax optimization of
       Jensen-Shannon divergence between discriminator performance on x_real vs. x_model.
       Uses ADAM optimizer for both networks, using params defined in DCGAN paper.
       Latent codes Z are drawn from a normal prior.
       
       Parameters
       - num_epochs: the number of training epochs over the dataset
       - dg_ratio: the ratio of discriminator batches to generator batches
       - batch_size: the train_set batch size passed into discriminator
                     (for each real batch, an equal-sized batch is generated by generator)
    """
    
    # Move to GPU if possible; batches get moved as needed to save on memory
    device = "cuda:0" if use_cuda and torch.cuda.is_available() else "cpu"
    
    criterion_dis = jensen_shannon("dis", device)
    criterion_gen = jensen_shannon("gen", device)
    criterion = nn.BCELoss()
    optimizer_dis = torch.optim.Adam(dis.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_gen = torch.optim.Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    loader = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True)
    # Keep a global count of how many discriminator steps happen per generator step
    # because dg_ratio may not divide evenly into batch_size
    dis_count = 0
    for epoch in range(num_epochs):
        #print(f"EPOCH {epoch+1}")
        
        for X, y in loader:
            z = torch.randn(X.shape[0], gen.z_len, 1, 1).to(device)
            X = X.clone().to(device, non_blocking=True)
            fake = torch.zeros(X.shape[0]).to(device)
            real = torch.ones(X.shape[0]).to(device)
            
            # Discriminator trains each batch
            optimizer_dis.zero_grad()
            #loss_dis = criterion_dis(dis(gen(z)), dis(X))
            #loss_dis.backward()
            dis_real = dis(X).view(-1)
            loss_dis_real = criterion(dis_real, real)
            loss_dis_real.backward()
            dis_fake = dis(gen(z).detach()).view(-1)
            loss_dis_fake = criterion(dis_fake, fake)
            loss_dis_fake.backward()
            loss_dis = loss_dis_real + loss_dis_fake
            optimizer_dis.step()
            dis_count += 1
            
            # Generator trains if enough discriminator passes have gone through
            if dis_count >= dg_ratio:
                #z = torch.randn(batch_size, gen.z_len).to(device)
                
                optimizer_gen.zero_grad()
                #loss_gen = criterion_gen(dis(gen(z)))
                dis_fake = dis(gen(z)).view(-1)
                loss_gen = criterion(dis_fake, real)
                loss_gen.backward()
                optimizer_gen.step()
                dis_count = 0
 
        # Evaluate how the model is performing on test set after a full epoch
        if (epoch+1) % print_every == 0:
            print(f"- EPOCH {epoch+1}:" +
                  f"\n  discriminator loss = {loss_dis}" +
                  f"\n      generator loss = {loss_gen}" +
                  "\n----------------------------------")

In [ ]:
gen = OnlineGenerator().cuda()
dis = OnlineDiscriminator().cuda()
train_set = CIFARDataset().load([f"data/data_batch_{n}" for n in range(1, 2)])
test_set = CIFARDataset().load(["data/test_batch"])

In [ ]:
%time train_DCGAN(gen, dis, train_set, test_set, dg_ratio=1, num_epochs=50, print_every=5)

In [ ]:
z_test = torch.randn(1000, gen.z_len, 1, 1).cuda()
gen_z_test = gen(z_test)
best_idx = torch.randperm(1000)[:16]

fig, ax = plt.subplots(4, 4, figsize=(8, 8))
for i in range(4):
    for j in range(4):
        plot_image(gen(z_test)[best_idx[4*i + j]], ax[i][j])
        #plot_image(train_set[1][0], ax[1])
plt.tight_layout()

#print(f"Fake images discriminator:\n{dis(gen(z_test[:10]))}\n")
#print(f"Real images discriminator:\n{dis(train_set[:10][0].cuda())}\n")
print(f"Possible labels: {[label.decode('ascii') for label in cifar_meta[b'label_names']]}\n")

In [ ]:
#torch.save(gen, "models/online_gen")
#torch.save(dis, "models/online_dis")

## Conditional DCGANs

As an attempt to stabilize the model, we implement a training procedure for the conditional version of the DCGAN architecture above, based on the original <a href="https://arxiv.org/pdf/1411.1784.pdf">Conditional GAN</a> paper and <a href="https://arxiv.org/pdf/1606.03498.pdf">Salimans, et al (2016)</a>.  Whereas the vanilla GAN optimizes the objective function
$$
\min_G \max_D V(G, D)
    = \mathbb{E}_{\mathbf{x} \sim p_{real}(\mathbf{x})} \log D(\mathbf{x})
    + \mathbb{E}_{\mathbf{z} \sim p_{model}(\mathbf{x})} \log (1-D(G(\mathbf{z})))
$$
the conditional GAN simply adjusts this to
$$
\min_G \max_D V(G, D)
    = \mathbb{E}_{\mathbf{x} \sim p_{real}(\mathbf{x})} \log D(\mathbf{x} | \mathbf{y})
    + \mathbb{E}_{\mathbf{z} \sim p_{model}(\mathbf{x})} \log (1-D(G(\mathbf{z} | \mathbf{y})))
$$
where $\mathbf{y}$ is some auxiliary vector (in this case, the CIFAR-10 labels).

This can be accomplished by simply concatenating the auxiliary vector $\mathbf{y}$ with a dense encoding in both the generator (at the input layer) and the discriminator (at the output layer).  The loss function Below is essentially a generalized version of the code above which allows for labels.  Setting `y_len = 0` and `num_classes = 1` recovers the unconditional model.

In [ ]:
# Conditional generalizations of some of the architecture/training functions
# http://cs231n.stanford.edu/reports/2015/pdfs/jgauthie_final_report.pdf

class CDCGenerator(DCGenerator):
    """CONDITIONAL deep convolutional generator which maps
       latent noise vector z, auxiliary y -> (32,32) RGB-channel image.
       The latent space input z is projected and convolved with many feature maps.
       Subsequent layers use only fractional-strided convolutions (no pooling)."""
    
    def __init__(self, z_len=100, y_len=10, code_len=5):
        super().__init__(z_len)
        self.y_len = y_len
        self.y_encode = nn.Linear(y_len, code_len)
        # Modify the parent architecture to accept larger input
        self.in_layer = nn.Linear(z_len + code_len, self.in_layer.out_features)
        
    def forward(self, z, y):
        # Encode y to avoid sparse representation
        y_code = self.y_encode(y)
        # Concatenate z and y, reshape as 4d tensor
        z = torch.cat([z, y_code], dim=1)
        z = self.in_layer(z).view(-1, 128, 4, 4)
        return self.conv_layers(z)
    

class CDCDiscriminator(DCDiscriminator):
    """CONDITIONAL deep convolutional discriminator which maps
       (32,32) RGB-channel image, auxiliary y -> [0, 1].
       The image is passed through several convolutional layers (again, no pooling).
       All hidden layers use LeakyReLU activation, and output layer uses sigmoid."""
    
    def __init__(self, leak_slope=0.02, y_len=10, code_len=5):
        super().__init__(leak_slope)
        self.y_len = y_len
        self.y_encode = nn.Linear(y_len, code_len)
        # Modify the parent architecture to allow concatenating in final layer
        self.out_layer[2] = nn.Linear(self.out_layer[2].in_features + code_len, 1)
        
    def forward(self, x, y):
        # Encode y to avoid sparse representation
        y_code = self.y_encode(y)
        x = self.conv_layers(x)
        # Reshape discriminator output by flattening
        x = x.view(-1, torch.prod(torch.tensor(x.shape[1:])))
        x = self.out_layer[:2](x)
        # Concatenate y at the final stage, before sigmoid activation
        x = torch.cat([x, y_code], dim=1)
        return self.out_layer[2:](x)


def onehot(y, num_classes, as_float=True):
    """Returns a one-hot encoding version of 1d vector y."""
    
    y_onehot = torch.zeros(len(y), num_classes, dtype=torch.long)
    y_onehot[range(len(y)), y] = 1
    if as_float:
        y_onehot = y_onehot.float()
    return y_onehot

    
def cond_jensen_shannon(which_model, device):
    """Returns a function to compute an empirical estimate of the Jensen-Shannon
       divergence between the true data-generating distribution P_real and
       the generated distribution P_model over parallel batches x_real and x_model,
       CONDITIONED on auxiliary vector y.
       
       This metric is optimized by a minimax operation, first maximizing over the
       discriminator weights, then minimizing over the generator weights.
       
       Returns:
       - if which_model = 'dis': returns function (D(x_model), D(x_real), y) -> scalar
       - if which_model = 'gen': returns function (D(x_model), y) -> scalar
         Both functions are returned in the form of minimization problems.
       - if which_model = 'full': returns non-negative version of 'dis' function.
         Use this version to calculate divergence after an epoch."""
    
    ce = nn.CrossEntropyLoss()
    
    # Functions to compute respective discriminator labels
    fake = lambda d: torch.zeros(d.size(), dtype=torch.long).to(device)
    
    if which_model == 'dis':
        #return lambda dis_model, dis_real: \
            #-torch.mean(torch.log(dis_real) + torch.log(1 - dis_model))  
        model_loss = lambda d_model, y: ce(d_model, 0 * y)
        real_loss = lambda d_real, y: ce(d_real, y)
        return lambda d_model, d_real, y: model_loss(d_model, y) + real_loss(d_real, y)
        
    elif which_model == 'gen':
        # If generated samples are rejected easily, log(1 - D(x_model)) saturates
        # (stays close to 0), so use an approximation to improve early training
        #return lambda dis_model: \
            #-torch.mean(torch.log(dis_model))
        return lambda d_model, y: ce(d_model, y)
   
    else:
        raise ValueError("Argument `which_model` is not a valid value")
        

def train_CDCGAN(gen, dis, train_set, test_set, num_classes=1,
                num_epochs=100, dg_ratio=1, batch_size=128,
                use_cuda=True, print_every=1):
    """Simultaneously trains generator and discriminator using minimax optimization of
       Jensen-Shannon divergence between discriminator performance on x_real vs. x_model.
       Uses ADAM optimizer for both networks, using params defined in DCGAN paper.
       Latent codes Z are drawn from a uniform prior.
       
       Parameters
       - num_epochs: the number of training epochs over the dataset
       - dg_ratio: the ratio of discriminator batches to generator batches
       - batch_size: the train_set batch size passed into discriminator
                     (for each real batch, an equal-sized batch is generated by generator)
    """
    
    # Move to GPU if possible; batches get moved as needed to save on memory
    device = "cuda:0" if use_cuda and torch.cuda.is_available() else "cpu"
    
    criterion_dis = jensen_shannon("dis", device)
    criterion_gen = jensen_shannon("gen", device)
    optimizer_dis = torch.optim.Adam(dis.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_gen = torch.optim.Adam(gen.parameters(), lr=0.0002, betas=(0.5, 0.999))
    
    loader = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True)
    # Keep a global count of how many discriminator steps happen per generator step
    # because dg_ratio may not divide evenly into batch_size
    dis_count = 0
    for epoch in range(num_epochs):
        #print(f"EPOCH {epoch+1}")
        
        for X, y in loader:
            z = torch.randn((X.shape[0], gen.z_len)).to(device)
            # Sample random y values from distribution of labels
            y_rand = np.random.choice(num_classes, size=X.shape[0])
            y_rand_onehot = onehot(y_rand, num_classes).float().to(device)
            
            X = X.clone().to(device, non_blocking=True)
            y = y.clone().to(device, non_blocking=True)
            y_onehot = onehot(y, num_classes).float().to(device)
            
            # Discriminator trains each batch
            optimizer_dis.zero_grad()
            loss_dis = criterion_dis(dis(gen(z, y_rand_onehot), y_rand_onehot),
                                     dis(X, y_onehot))
            loss_dis.backward()
            optimizer_dis.step()
            dis_count += 1
            
            # Generator trains if enough discriminator passes have gone through
            if dis_count >= dg_ratio:
                z = torch.randn((X.shape[0], gen.z_len)).to(device)
                
                optimizer_gen.zero_grad()
                loss_gen = criterion_gen(dis(gen(z, y_rand_onehot), y_rand_onehot))
                loss_gen.backward()
                optimizer_gen.step()
                dis_count = 0
 
        # Evaluate how the model is performing on test set after a full epoch
        if (epoch+1) % print_every == 0:
            print(f"- EPOCH {epoch+1}:" +
                  f"\n  discriminator loss = {loss_dis}" +
                  f"\n      generator loss = {loss_gen}" +
                  "\n----------------------------------")

In [ ]:
cgen = CDCGenerator(y_len=len(label_set))
cdis = CDCDiscriminator(y_len=len(label_set))

z_test = torch.randn(15, cgen.z_len)
y_rand = np.random.choice(len(label_set), size=15)
X, y = train_set[:15]
y_onehot = onehot(y, cgen.y_len)

cdis(cgen(z_test, y_onehot), y_onehot).shape

In [ ]:
train_set = CIFARDataset().load([f"data/data_batch_{n}" for n in range(1, 2)])
test_set = CIFARDataset().load(["data/test_batch"])
label_set = [label.decode('ascii') for label in cifar_meta[b'label_names']]

cgen = CDCGenerator(y_len=len(label_set)).cuda()
cdis = CDCDiscriminator(y_len=len(label_set)).cuda()

In [ ]:
%time train_CDCGAN(cgen, cdis, train_set, test_set, num_classes=len(label_set), \
                   dg_ratio=1, num_epochs=50)

In [ ]:
z_test = torch.randn(1, cgen.z_len).cuda()
X, y = train_set[:1]
y_onehot = onehot(y, cgen.y_len).cuda()
fake_X = cgen(z_test, y_onehot)
fake_y = torch.randperm(cgen.y_len)[0]

fig, ax = plt.subplots(1, 2, figsize=(8, 4))
plot_image(fake_X[0], ax[0])
plot_image(X[0], ax[1])
ax[0].set_title(f"{label_set[fake_y.item()]}?")
ax[1].set_title(label_set[y[0].item()])

print(f"Possible labels: {[label.decode('ascii') for label in cifar_meta[b'label_names']]}\n")

## Evaluation of Generated Images

The main metric we use to evaluate how closely the generated images reflect the training set is the *Frechet inception distance.*  This is a distance metric which compares statistics of samples drawn from the $P_{model}$ and $P_{real}$ distributions.  For corresponding sets of samples $\{x^{(1)}_{model}, ..., x^{(N)}_{model}\}$ and $\{x^{(1)}_{real}, ..., x^{(N)}_{real}\}$, each $x^{(i)}$ is projected into a 2048-dimensional feature space by running a forward pass through an <a href="https://arxiv.org/pdf/1512.00567.pdf">Inception v3</a> convolutional net pre-trained on the ImageNet dataset, which here is acting essentially as an autoencoder.  This network maps tensors of size $(N, 3, 299, 299) \to (N, 1, 1, 100)$.  So the CIFAR-10 images have to first be zero-padded to the $(299, 299)$ ImageNet size, and then we have to slightly modify the model to output the penultimate layer of size $(N, 1, 1, 2048)$.

These embedded vectors are assumed to obey some multivariate Gaussian distribution, such that $x^{(i)}_{emb, model} \overset{\text{iid}}{\sim} \mathcal{N}_{2048}(\mu_1, \Sigma_1)$ and $x^{(i)}_{emb, real} \overset{\text{iid}}{\sim} \mathcal{N}_{2048}(\mu_2, \Sigma_2)$.  Then, the FID is the statistical Frechet distance between these two vectors:
$$ FID = ||\mu_1 - \mu_2||^2_2 + tr[\Sigma_1 + \Sigma_2 - 2(\Sigma_1 \Sigma_2)^{1/2}] $$

In [ ]:
from torchvision.models import inception_v3
from scipy.linalg import sqrtm
from torch.nn.modules.upsampling import Upsample
from similaritymeasures import frechet_dist

iv3 = inception_v3(pretrained=True, aux_logits=False)
# Remove the final layer from the forward pass
iv3.fc = nn.Identity()
# Add a non-weighted upsampling layer to make this compatible to CIFAR-10
upsample = Upsample(scale_factor=299/32)

# FID runs considerably faster if this is loaded on GPU
if torch.cuda.is_available():
    iv3.cuda()

In [ ]:
def calc_fid(fake_set, real_set, num_samples=1000, batch_size=128):
    """Calculates the Frechet Inception Distance (FID) between two empirical distributions.
       Both `fake_set` and `real_set` should be CIFARDatasets which can be data loaded.
       Computes distance over min{`num_samples`, min{len(fake_set), len(real_set)}}
       random samples drawn from respective datasets.
       
       It really helps to run this on a GPU, if possible."""
    
    min_len = min(len(fake_set), len(real_set))
    num_samples = min(num_samples, min_len)
    random_idx = torch.randperm(min_len)[:num_samples]
    
    # Pick some random layer of Inception v3 to check device
    device = iv3.Conv2d_1a_3x3.conv.weight.device
    
    embeds = [torch.empty(num_samples, 2048), torch.empty(num_samples, 2048)]
    
    for dataset, i in zip([fake_set, real_set], [0, 1]):
        loader = torch.utils.data.DataLoader(CIFARDataset(*dataset[random_idx]), batch_size)
        
        # Run the encoding in batches to avoid memory overload
        for batch_num, (samples, _) in enumerate(loader):
            # Necessary to stop gradients from accumulating in iv3 forward passes
            with torch.no_grad():
                samples = samples.to(device)
                # (N, 3, 32, 32) -> (N, 2048)
                embedded = iv3(upsample(samples))
                samples = samples.cpu()
                embeds[i][batch_num*batch_size:(batch_num+1)*batch_size, :] = embedded   
    
    # Use pre-compiled discrete Frechet distance function
    return frechet_dist(embeds[0], embeds[1])